In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
import sys

sys.path.insert(0, "python")
from network import *
from protocols import *
from adversary import *
from simulator import *
from experiments import run_and_eval, run_experiment

In [3]:
nw_generator = NodeWeightGenerator("random")
ew_generator = EdgeWeightGenerator("normal")

# Dandelion example with random seed

In [4]:
seed = 42

from data import GoerliTestnet
net = GoerliTestnet()
net = Network(nw_generator, ew_generator, graph=net.graph, seed=seed)

In [5]:
net = Network(nw_generator, ew_generator, num_nodes=1000, k=50, seed=seed)

In [6]:
dp = DandelionProtocol(net, 0.5, seed=seed)#, broadcast_mode='all')

dp = DandelionPlusPlusProtocol(net, 0.5, seed=seed)#, broadcast_mode='all')

nx.draw(dp.anonymity_graph,  with_labels = True)

adv = Adversary(dp, 0.1, seed=seed)

adv_nodes = net.get_central_nodes(int(net.num_nodes*0.1), "degree")
adv = Adversary(dp, adversaries=adv_nodes)

In [7]:
adv = DandelionAdversary(dp, 0.1, seed=seed)

In [8]:
sim = Simulator(adv, 10, seed=seed, verbose=False)

In [9]:
%%time
_ = sim.run()

CPU times: user 1.59 s, sys: 17.6 ms, total: 1.61 s
Wall time: 1.61 s


In [10]:
%%time
evaluator = Evaluator(sim, "first_reach")
print(evaluator.get_report())

{'estimator': 'first_reach', 'hit_ratio': 0.2, 'inverse_rank': 0.31111195654904034, 'entropy': 2.5975998532287106, 'ndcg': 0.41731343626903483, 'message_spread_ratio': 1.0}
CPU times: user 114 ms, sys: 7.67 ms, total: 122 ms
Wall time: 121 ms


In [11]:
%%time
evaluator = Evaluator(sim, "first_sent")
print(evaluator.get_report())

{'estimator': 'first_sent', 'hit_ratio': 0.2, 'inverse_rank': 0.33600340634669007, 'entropy': 2.801717150959715, 'ndcg': 0.44920545773306547, 'message_spread_ratio': 1.0}
CPU times: user 130 ms, sys: 958 µs, total: 131 ms
Wall time: 130 ms


In [12]:
%%time
evaluator = Evaluator(sim, "dummy")
print(evaluator.get_report())

{'estimator': 'dummy', 'hit_ratio': 0.0, 'inverse_rank': 0.005353680457593244, 'entropy': 9.813781191217043, 'ndcg': 0.12628442924180622, 'message_spread_ratio': 1.0}
CPU times: user 37.9 ms, sys: 0 ns, total: 37.9 ms
Wall time: 37.2 ms


# TOREP first steps

rng = np.random.default_rng(32)
G = nx.random_regular_graph(3,10,seed=rng)
print(list(G.neighbors(0)))
nx.draw(G)

rng = np.random.default_rng(32)
rng.choice(G.nodes(), 3)

net = Network(nw_generator, ew_generator, num_nodes=1000, k=20)

seed = 41
net = Network(nw_generator, ew_generator, num_nodes=10, k=2, seed=seed)

np.mean(list(dict(net.graph.degree).values()))

tor = TOREnhancedProtocol(net, 1, 2, broadcast_mode="all", seed=seed)

np.mean(list(dict(net.graph.degree).values()))

tor.tor_network

msg = Message(0)
adv = Adversary(tor, 0.3, seed=seed)

adv.nodes

msg.queue

msg.process(tor, adv)

list(net.graph.neighbors(4))